<a href="https://colab.research.google.com/github/natakatch/Springboard/blob/main/hate_speach_pre_processing_training_data_development_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CAPSTONE Three.

# Hate Speech and Offensive Language detection

## Pre-processing and Training Data Development, Modeling

## Introduction

Text classification is one of the major problems people are now days looking to solve. We will use supervised machine learning algoritms and NLP modeling for our hate-speech detection research; we will be using Twitter data. 
The text is already classified as: 
 * hate-speech, 
 * offensive language, 
 * and neither. 

Word embeddings (algorithm for obtaining vector representations for words)
  - Supervised with NLP - bag of words of models od tf-ids features
  - Unsupervised learning GloVe.

- Use Multinomial Naive Bayes algorithm is a probabilistic learning method 
- Random Forest
- 
- Use Bert in multi label Text Classification

## Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from scipy import stats
# random enables us to generate random numbers
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.model_selection import train_test_split



In [ ]:
from sklearn import tree, metrics
from io import StringIO
from IPython.display import Image  
import pydotplus

from sklearn import preprocessing

from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import accuracy_score,log_loss
from sklearn.metrics import confusion_matrix, accuracy_score, plot_confusion_matrix, classification_report


In [ ]:

import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
#from bs4 import BeautifulSoup
import unicodedata

from nltk.tokenize import TweetTokenizer


In [ ]:
import sys
import torch

from torch import nn

##  Load the data

In [ ]:
from google.colab import files


uploaded = files.upload()


Load the Hate Speach and Offensive language dataset (labeled_data.csv)

In [ ]:
hate_speach = pd.read_csv('labeled_data.csv')

In [ ]:
hate_speach

In [ ]:
hate_speach.info()

In [ ]:
hate_speach.shape

 # 4. Pre-processing and Training Data Development

The text is classified as: hate-speech, offensive language, and neither.
* hate speech it is labelled as 0
* offensive it is labelled as 1
* neither then it is class 2

The dataset has 7 columns. We will drop all the columns that contailn numbers of CF users who judged the tweet and keep only 2 columns: the tweet and class

In [ ]:
hs = hate_speach[['class', 'tweet']]
hs

In [ ]:
# Grouping data by label
hs.groupby('class').count()

In [ ]:
 def percentage(part, whole):
  Percentage = round(100 * float(part)/float(whole),2)
  return str(Percentage) + "%"

In [ ]:
hate, offensive, neither = np.bincount(hs['class'])
whole = hate + offensive + neither


In [ ]:
hate_pr = percentage(hate,whole)
offensive_pr = percentage(offensive,whole)
neither_pr = percentage(neither,whole)

In [ ]:
print("Total : " + str(whole))
print()
print("Hate : " + hate_pr + " of total")
print("Offensive : " + offensive_pr + " of total")
print("Neither : " + neither_pr + " of total")

This is an unbalanced dataset.

In [ ]:
import seaborn as sn
sn.distplot(hs['class'])
plt.show()

In [ ]:
hs['class'].hist()

### 4.1 Create dummy or indicator features for categorical variables

In [ ]:
tweet = hs.drop('class', axis=1)
categories = hs['class']

Although some machine learning algorithms can interpret multi-level categorical variables, many machine learning models cannot handle categorical variables unless they are converted to dummy variables.

In [ ]:
cat = pd.get_dummies(categories)

where hate speech it is labelled as 0, offensive it is labelled as 1 
neither then it is class 2

In [ ]:
cat.describe()

In [ ]:
df = pd.concat([hs.drop("class", axis=1), pd.get_dummies(categories)], axis=1)
df

## Preprocessing of the tweets

* Removal of punctuation and capitlization
* Tokenizing
* Removal of stopwords
* Stemming

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


In [ ]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>@')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


In [ ]:
df['cleanTweet']=df['tweet'].map(lambda s:preprocess(s)) 
tweet_tokenizer = TweetTokenizer()
df['tokenized_tweets'] = df['cleanTweet'].apply(tweet_tokenizer.tokenize)
df

In [ ]:
df = df.drop("tweet", axis=1)
df

### Save clean tweens in a new file

In [ ]:
#df.to_csv('../data/clean_tweets.csv') 

In [ ]:
hs = hs.assign(cleanTweet=hs['tweet'].map(lambda s:preprocess(s))) 


In [ ]:
hs = hs.drop("tweet", axis=1)
hs

In [ ]:
hs.rename(columns={'class' : 'label'}, inplace=True)


### 4.2. Standardize the magnitude of numeric features

This is not applied in our situation. We don't have numeric features with different magnitude. 

## Visualization . Word Clouds 

Word Clouds are a popular way of displaying how important words are in a collection of texts. 
The more frequent the word is, the greater space it occupies in the image. 

### Offensive language

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
stopwords.add("RT")

print(type(STOPWORDS))

import random

def random_color_func(word=None, font_size=None, position=None,  orientation=None, font_path=None, random_state=None):
    h = 250
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)
    return "hsl({}, {}%, {}%)".format(h, s, l)

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=60, 
                          random_state=42
                         ).generate(str(hs.loc[hs["label"]==1].cleanTweet))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud.recolor(color_func= random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis('off')
plt.show()

### Hate language

In [ ]:
stopwords.add("Name")

def random_color_func(word=None, font_size=None, position=None,  orientation=None, font_path=None, random_state=None):
    h = 344
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)
    return "hsl({}, {}%, {}%)".format(h, s, l)

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=60, 
                          random_state=42
                         ).generate(str((hs.loc[hs["label"]==0].cleanTweet)))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud.recolor(color_func= random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis('off')
plt.show()

### Neither


In [ ]:
stopwords.add("Name")

def random_color_func(word=None, font_size=None, position=None,  orientation=None, font_path=None, random_state=None):
    h = 120
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)
    return "hsl({}, {}%, {}%)".format(h, s, l)

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=60, 
                          random_state=42
                         ).generate(str((hs.loc[hs["label"]==2].cleanTweet)))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud.recolor(color_func= random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis('off')
plt.show()

# 5 Modeling

### Word embeddings

#### Bag of words and Tf-idf

Use Bag-of-words vectors (build in scikit-learn) to predict the text label(category) based on the tweets wording.

The data set we extracted has categorical features we generated using preprocessing

The BoW method is simple and works well, but it treats all words equally and cannot distinguish very common words or rare words. Tf-idf solves this problem of BoW Vectorization.

Both Vectorization techniques, BoW and tf-idf work well but it fails to suggest a relation between two words. Vectorization using word embedding solves this problem. We will discuss word embedding 

In [ ]:
hs
#hs.groupby('label').count()

In [ ]:
#target
y = hs.label

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(hs["cleanTweet"], y,test_size=0.25, random_state=0)

#### CountVectorizer for text classification

In [ ]:
count_vectorizer = CountVectorizer(stop_words="english")

In [ ]:
# Transform the training data using only the 'tweet' column values: count_train 
count_train = count_vectorizer.fit_transform(X_train)

# Transform the test data using only the 'text' column values: count_test 
count_test = count_vectorizer.transform(X_test)

#### TfidfVectorizer for text classification

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)


### Inspecting the vectors

In [ ]:
# tfidf_vectorizer the first 10 features
tfidf_vectorizer.get_feature_names()[:10]

In [ ]:
# The first 5 vectors of the tfidf training data
tfidf_train.A[:5]

#### Create the CountVectorizer and TfidfVectorizer dataframes

In [ ]:
count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names() )

#### CountVectorizer

In [ ]:
count_df.head()

#### TfidfVectorizer

In [ ]:
tfidf_df.head()

### Calculate the difference in columns

In [ ]:
difference = set(count_df.columns) - set(tfidf_df.columns)
difference

### Check whether the DataFrames are equal

In [ ]:
count_df.equals(tfidf_df)

In [ ]:
hs

## Naive Bayes model

Commonly used when training a new supervised model using text vector data


#### Train and test a Naive Bayes model using the CountVectorizer data.

In [ ]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

# Instantiate a Multinomial Naive Bayes classifier: nb_classifier
nb_classifier = MultinomialNB()

# Fit the classifier to the training data
nb_classifier.fit(count_train, y_train)

# Create the predicted tags: pred
pred = nb_classifier.predict(count_test)



#### Calculate the accuracy score

In [ ]:
score = metrics.accuracy_score(y_test, pred)
score

#### Calculate the confusion matrix

In [ ]:
cm = metrics.confusion_matrix(y_test,pred, labels=[0, 2])
cm

In [ ]:
plt.figure(figsize = (7,5))

sns.heatmap(cm, annot=True, cmap ='Greens')

#### Train and test a Naive Bayes model using the TfidfVectorizer

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(tfidf_train,y_train)

# Create the predicted tags: pred
pred = nb_classifier.predict(tfidf_test)


In [ ]:
# Calculate the accuracy score
accuracy = metrics.accuracy_score(y_test, pred)
accuracy

In [ ]:
# Calculate the confusion matrix: cm
cm = metrics.confusion_matrix(y_test, pred, labels=[0, 2])
cm

In [ ]:
plt.figure(figsize = (7,5))
sns.heatmap(cm, annot=True, cmap ='Greens')

## Classification with Random Forest

The random forest algorithm works by completing the following steps:

    * select random samples from the dataset provided.
    * create a decision tree for each sample selected. Then it will get a prediction result from each decision tree created.
    * voting will then be performed for every predicted result. For a classification problem, it will use mode, and for a regression problem, it will use mean.
    * select the most voted prediction result as the final prediction.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(hs["cleanTweet"], y,test_size=0.25, random_state=0)

In [ ]:
classifier_rf = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier_rf.fit(count_train, y_train)

In [ ]:
y_pred_rf = classifier_rf.predict(count_test)
cm = confusion_matrix(y_test, y_pred_rf)
cm

In [ ]:
plt.figure(figsize = (7,5))
sns.heatmap(cm, annot=True, cmap ='Blues')

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_pred_rf)
accuracy

# Deep neural network for hate speech detection

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

In [ ]:
df

Tweets cannot be directly interpreted we need to transform them in a sequence of meaningful integer number in order to be analyzed by NN model

## GloVe Word Embeddings with Keras

In [ ]:
from scipy import spatial
from sklearn.manifold import TSNE

Loading Glove Pre-trained Word Embedding Model from Text File


GloVe embeddings were trained on a dataset of one billion words with a vocabulary of 400 thousand words. There are few different embedding vector sizes such as 50, 100, 200 and 300 dimensions. We use 50 dimentions vector

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.cleanTweet)
print('Found %d unique words.' % len(tokenizer.word_index))

In [ ]:
X = tokenizer.texts_to_sequences(df.cleanTweet)
X[:5] # first 5 elements of the X 

In [ ]:
y = df[2] # column 2 - no hate or offensive language (0s - hate or offensive language)
y.shape

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(X[0])
print(X[1])
print()
print("Max lenght of X")
max_length = max([len(s) for s in X]) #maximum length of all sequences. If not provided, sequences will be padded to the length of the longest individual sequence
print(max_length)
print()
padded_seq = pad_sequences(X,maxlen=max_length,padding='post')
print(padded_seq[0])
print(padded_seq[1])


Preparing Training and Test dataset

In [ ]:
padded_seq,y = np.array(padded_seq),np.array(y)

X_train,X_test,Y_train,Y_test = train_test_split(padded_seq,y,test_size=0.25,stratify=y)

In [ ]:
print (X_train.shape)
print(y_train.shape)
print (X_test.shape)
print(y_test.shape)

In [ ]:
MAX_NUM_WORDS = 100000
EMBEDDING_DIM = 50
MAX_SEQUENCE_LENGTH = 200
VALIDATION_SPLIT = 0.2
word_index = tokenizer.word_index

len(word_index)


In [ ]:
test_data = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
test_data.shape 

In [ ]:
embeddings_index = dict()
f = open('glove.6B.50d.txt',mode='rt',encoding='utf-8')
import numpy as np
for line in f:
    values = line.split()
    words = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[words] = coefs
f.close()
print('Loaded word vectors',len(embeddings_index)) #Loaded word vectors 400000

In [ ]:


# prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from glove.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector



In [ ]:
embedding_matrix

In [ ]:
embedding_matrix.shape

## Simple NN model

After the  pre-processing, the final dataset is ready to be analyzed. Due to the low dimensionality of the dataset, a simple NN model, with just an LSTM layer with 10 hidden units, will suffice the task:

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from tensorflow.keras.initializers import Constant

# specify the maximum input length to the Embedding layer.
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') 

# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print("Preparing of embedding matrix is done")

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM


In [ ]:
rnnmodel = Sequential()
#rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(embedding_layer)
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(2, activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])




In [ ]:
rnnmodel.fit(X_train, Y_train,
          batch_size=32, 
          epochs=1,
          validation_split=0.2)

score, acc = rnnmodel.evaluate(X_test, Y_test,
                            batch_size=32)
print('Test accuracy with RNN:', acc)